## CostPathFromSurface Workflow

In [21]:
from CostPathFromSurface import *

In [ ]:
 # Input Parameters
workSpaceGDB = 'C:\\Users\\Eric Kerney\\arcgisNotebooks\\dronecode\\Surface_v2.gdb\\Surface.gdb'
surfaceFile, originLoc, destLoc, siteName = 'PendletonOR_SuitabilitySurface', 'TribalCenter', 'Pendleton_Lab', 'pendleOR'
inPrj = 'GEOGCS[\"GCS_WGS_1984\",DATUM[\"D_WGS_1984\",SPHEROID[\"WGS_1984\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]]'
outPrj = 'PROJCS[\"US_National_Atlas_Equal_Area\",GEOGCS[\"GCS_Sphere_Clarke_1866_Authalic\",DATUM[\"D_Sphere_Clarke_1866_Authalic\",SPHEROID[\"Sphere_Clarke_1866_Authalic\",6370997.0,0.0]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Lambert_Azimuthal_Equal_Area\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-100.0],PARAMETER[\"Latitude_Of_Origin\",45.0],UNIT[\"Meter\",1.0]]'
surfaceAttr, rastCellSize, simplifyTol = 'score_v2', '0.0004','300 Meters'
# Instantiate Class
surfacePath = CostPathFromSurface(workSpaceGDB, surfaceFile, originLoc, destLoc, siteName, inPrj, outPrj, surfaceAttr, rastCellSize, simplifyTol)
# Run Process
surfacePath.runGeoTool()

### RouteToKML Workflow

In [1]:
from RouteToKML import *

In [ ]:
# Input parameters
inputData = 'pendleOR_GeoJSON_13042022132656.geojson'
z_units = 'ft'
agl = 400
outputName = 'PendletonRoutes'
# Instantiate RouteToKML class
routes = RouteToKML(inputData, z_units, agl, outputName)
# Run Process
routes.runGeoTool()

## Waypoints To GeoJSON

In [ ]:
from WaypointsProcess import *

In [40]:
filePath = '1727592874210492416.waypoints'
outputName = 'SampleWaypoints'
columnFormat = [(0,2), (2,4), (4,6), (6,9), (9,11), (11,13), (13,15), (15,18), (18,31), (31,44), (44,56), (56,57)]
columnNames = ['INDEX', 'CURRENT', 'COORD_FRAME', 'COMMAND', 'PARAM1', 'PARAM2','PARAM3','PARAM4', 
               'PARAM5/X/LATITUDE','PARAM6/Y/LONGITUDE','PARAM7/Z/ALTITUDE','AUTOCONTINUE' ]
lonCol, latCol, zCol = 'PARAM6/Y/LONGITUDE', 'PARAM5/X/LATITUDE', 'PARAM7/Z/ALTITUDE'
dotWaypoints = Waypoints(filePath, outputName, columnFormat, columnNames, lonCol, latCol, zCol)
dotWaypoints.dfToGeoJSON()
dotWaypoints.saveGeoJSON()

Loaded: 1727592874210492416.waypoints as DataFrame
Created GeoJSON dictonary from dataframe
GeoJSON output: SampleWaypoints_14042022120227.geojson


In [ ]:
from WaypointsProcess import *

In [ ]:
# Input parameters
inputData = 'pendleOR_GeoJSON_13042022132656.geojson'
z_units = 'm'
agl = 30.48	
outputName = 'waypointsTest'
# Instantiate RouteToWaypoints class
routes = RouteToWaypoints(inputData, z_units, agl, outputName)
# Run Process
df = routes.runGeoTool()

## OTHER STUFF

In [39]:
import pandas as pd
import time
import json

class Waypoints:

    def __init__(self, filePath: str, outputName: str, columnFormat: list, columnNames: list, 
                 lonCol: str, latCol: str, zCol: str):
        self.colSpecs, self.colNames, self.filePath, self.outputName = columnFormat,columnNames,filePath,outputName 
        self.lon, self.lat, self.z = lonCol, latCol, zCol
        self.df = pd.read_fwf(self.filePath, skiprows=1, colspecs=self.colSpecs, names=self.colNames)
        #self.df = df[(df[lonCol] != 0) | (df[latCol] != 0)]
        print(f'Loaded: {filePath} as DataFrame')

    def dfToGeoJSON(self):
        # example from https://notebook.community/captainsafia/nteract/applications/desktop/example-notebooks/pandas-to-geojson
        # empty python dictonary for GeoJSON
        self.geojson = {'type':'FeatureCollection', 'features':[]}
        for _, row in self.df.iterrows():
            # template for each feature
            feature = {'type':'Feature',
                        'properties':{},
                    'geometry':{'type':'Point',
                                'coordinates':[]}}
            feature['geometry']['coordinates'] = [ row[self.lon], row[self.lat], row[self.z] ]
            # add each column as GeoJSON property
            for prop in self.colNames:
                feature['properties'][prop] = row[prop]
            # append feature to Feature Collection
            self.geojson['features'].append(feature)
        print(f'Created GeoJSON dictonary from dataframe')

    def saveGeoJSON(self):
        # serialize GeoJSON object
        self.json_object = json.dumps(self.geojson, indent = 4)
        # create file timestamp
        self.timeStamp = time.strftime('%d%m%Y%H%M%S')
        # write GeoJSON output file
        with open((f'{self.outputName}_{self.timeStamp}.geojson'), "w") as outfile:
            outfile.write(self.json_object)
        print(f'GeoJSON output: {self.outputName}_{self.timeStamp}.geojson')


In [ ]:
with open('1727592874210492416.waypoints') as f:
    for line in f:
        print(line.strip())

In [7]:
colspecs = [(0,2), (2,4), (4,6), (6,9), (9,11), (11,13), (13,15), (15,18), (18,31), (31,44), (44,56), (56,57)]
df = pd.read_fwf('1727592874210492416.waypoints', skiprows=1, colspecs=colspecs,
    names=['INDEX', 'CURRENT', 'COORD_FRAME', 'COMMAND', 'PARAM1', 'PARAM2','PARAM3','PARAM4',
    'PARAM5/X/LATITUDE','PARAM6/Y/LONGITUDE','PARAM7/Z/ALTITUDE','AUTOCONTINUE' ] )
df

,INDEX,CURRENT,COORD_FRAME,COMMAND,PARAM1,PARAM2,PARAM3,PARAM4,PARAM5/X/LATITUDE,PARAM6/Y/LONGITUDE,PARAM7/Z/ALTITUDE,AUTOCONTINUE
0,0,0,0,16,0,0,0,0.0,34.048092,-84.092349,30.48,1
1,1,0,0,22,0,0,0,0.0,0.000000,0.000000,30.48,0
2,2,1,0,16,0,0,0,0.0,34.045665,-84.093778,30.48,1
3,3,2,0,16,0,0,0,0.0,34.044154,-84.095687,30.48,1
4,4,3,0,16,0,0,0,0.0,34.043496,-84.096431,30.48,1
5,5,4,0,16,0,0,0,0.0,34.044003,-84.097450,30.48,1
6,6,5,0,16,0,0,0,0.0,34.044403,-84.099070,30.48,1


In [8]:
dfGeo = df[df['PARAM6/Y/LONGITUDE'] != 0]
dfGeo

,INDEX,CURRENT,COORD_FRAME,COMMAND,PARAM1,PARAM2,PARAM3,PARAM4,PARAM5/X/LATITUDE,PARAM6/Y/LONGITUDE,PARAM7/Z/ALTITUDE,AUTOCONTINUE
0,0,0,0,16,0,0,0,0.0,34.048092,-84.092349,30.48,1
2,2,1,0,16,0,0,0,0.0,34.045665,-84.093778,30.48,1
3,3,2,0,16,0,0,0,0.0,34.044154,-84.095687,30.48,1
4,4,3,0,16,0,0,0,0.0,34.043496,-84.096431,30.48,1
5,5,4,0,16,0,0,0,0.0,34.044003,-84.097450,30.48,1
6,6,5,0,16,0,0,0,0.0,34.044403,-84.099070,30.48,1


In [9]:
#https://notebook.community/captainsafia/nteract/applications/desktop/example-notebooks/pandas-to-geojson
def df_to_geojson(df, properties, lat='PARAM5/X/LATITUDE', lon='PARAM6/Y/LONGITUDE'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}
    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [12]:
cols =['INDEX', 'CURRENT', 'COORD_FRAME', 'COMMAND', 'PARAM1', 'PARAM2','PARAM3','PARAM4',
       'PARAM5/X/LATITUDE','PARAM6/Y/LONGITUDE','PARAM7/Z/ALTITUDE','AUTOCONTINUE']
geojson = df_to_geojson(dfGeo, cols)

In [26]:
# Serializing json 
json_object = json.dumps(geojson, indent = 4)
timeStamp = time.strftime('%d%m%Y%H%M%S')
with open((f'Waypoints_{timeStamp}.geojson'), "w") as outfile:
    outfile.write(json_object)

In [ ]:
# Input parameters
inputData = 'Waypoints_07042022103809.geojson'
z_units = 'm'
agl = 30.48	
outputName = 'waypointsTest'
# Instantiate RouteToKML class
routes = RouteToWaypoints(inputData, z_units, agl, outputName)
# Run Process
routes.runGeoTool()

## GeoJSON => Waypoints

## RouteToWaypoints

In [9]:
# install dependencies and python package install function for python-dotenv
import subprocess
import sys
import os
import requests
import json
import pandas as pd

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])    
install('python-dotenv')

class RouteToWaypoints:
    '''This class ingests simplified GeoJSON output from the :class: 'CostPathFromSurface' class,
    first loading the GeoJSON LineString file, then adding CARS elevation data to points via Airspace Link 
    Azure function, next calculating the altitude from the CARS attributes and AGL parameter, then generating 
    a .waypoints file from the leastCostPath in QGroundControl filw version - QGC WPL 110
    Dependencies: requests, json, pandas, subprocess, sys, os, python-dotenv
    python-dot is required to load client credentials from dotenv file in project root directory
    The tab separated attributes in waypoints export are in the following order:
    'INDEX', 'CURRENT', 'COORD_FRAME', 'COMMAND', 'PARAM1', 'PARAM2', 'PARAM3', 'PARAM4', 'PARAM5/X/LATITUDE',
    'PARAM6/Y/LONGITUDE', 'PARAM7/Z/ALTITUDE', 'AUTOCONTINUE'
    :param geoJSONpath: path to GeoJSON file
    :type geoJSONpath: str
    :param z_units: ft or m 
    :type z_units: str
    :param agl: flight altitude Above Ground Level, units specified above
    :type agl: float
    :param outName: waypoints output filename
    :type outName: str

    # Example Workflow
    inputData = 'SampleRoutes_v2.geojson'
    z_units = 'ft'
    agl = 400
    outputName = 'PendletonRoutes'
    routes = RouteToWaypoints(inputData, z_units, agl, outputName)
    routes.runGeoTool()
    ''' 
    
    def __init__(self, geoJSONpath: str, z_units: str, agl: float, outName: str,
                 featureType: str='LINESTRING', in_prj: str ='EPSG:4269', in_type: str='ellipsoid',):
        '''Constructor method
        '''
        self.inputPath, self.z_units, self.agl, self.featureType = geoJSONpath, z_units, agl, featureType
        self.in_prj, self.in_type, self.outName = in_prj, in_type, outName
        self.runGetToken()

    def runGetToken(self):   
        def get_token(client_id: str, client_key: str, scope: str, subscription_key: str) -> str:
            """
            Get an oauth token from the api
            :param client_id: Client ID
            :param client_key: Client Secret
            :param scope: Oauth Scope
            :param subscription_key: Subscription ID
            :return: Bearer Token
            :raises SystemError: System error if unable to get token
            """
            token_body = {
                "grant_type": "client_credentials",
                "client_id": client_id,
                "client_secret": client_key,
                "scope": requests.utils.quote(scope),
            }
            header = {
                "x-api-key": subscription_key,
                "Content-Type": "application/x-www-form-urlencoded",
            }
            response = requests.post(
                "https://airhub-api.airspacelink.com/v1/oauth/token",
                data=token_body,
                headers=header,
            )
            if response.status_code != 200:
                raise SystemError(
                    f"Unable to get oauth token due to: ({response.status_code}) {response.json()['message']}"
                )
            return response.json()["data"]["accessToken"]

        # install and load .env
        install('python-dotenv')
        from dotenv import load_dotenv 
        load_dotenv()
        # load env keys
        client_key = os.getenv('CLIENT_SECRET')
        client_id = os.getenv('CLIENT_ID')
        api_scope = os.getenv('API_SCOPE')
        subscription = os.getenv('SUBSCRIPTION')
        url = os.getenv('URL')
        # obtain token for CARS API request
        token = get_token(client_id, client_key, api_scope, subscription)
        self.subscription_key = subscription
        self.token = token
        print(f'Obtained token for Client ID: {client_id}')

    def runGeoTool(self):
        '''Execute all methods in workflow:
        self.loadGeoJSON()          # Loads GeoJSON file generated from :class:'CostPathFromSurface' class
        self.addCARSdata()          # Takes loaded GeoJSON and sends to Airspace Link CARS function which adds elevation attributes
        self.addAglGeoJSON()        # With geoJSONarr from CARS function, calculate flight altitude
        self.exportToWaypoints()    # Convert GeoJSON Feature Collection(s) into waypoints export as tab sep text file
        '''        
        self.loadGeoJSON()
        self.addCARSdata()
        self.addAglGeoJSON()
        return self.exportToWaypoints()

    # Load raw GeoJSON file as JSON     
    def loadGeoJSON(self):
        '''Loads GeoJSON file generated from :class:'CostPathFromSurface' class
        :return: Prints success message with name of loaded GeoJSON e.g.
        Loaded GeoJSON: pendleOR_GeoJSON_17032022144129.geojson
        :rtype: none
        '''         
        f = open(self.inputPath)
        self.loadedGeoJSON = json.load(f)
        print(f'Loaded GeoJSON: {self.inputPath}')

    # send in memory GeoJSON to CARS
    def addCARSdata(self):
        '''Takes loaded GeoJSON and sends to Airspace Link CARS function which adds elevation attributes, 
        :return: Prints success message with provided flight AGL e.g.
        CARS Data Request Success Flight AGL: 400 ft
        :rtype: none
        '''  
        self.geoJSONarr = []
        url = 'https://airhub-api.airspacelink.com/v1/elevation'
        headers = { 
            'Content-Type': 'application/json',
            'x-api-key': self.subscription_key,
            'Authorization': (f'Bearer {self.token}'),
            }
        # Iterate through list of GeoJSON features
        for i, feature in enumerate(self.loadedGeoJSON['features']):
            payload = json.dumps({
                "inVDatum": self.in_type,
                # "in_prj": self.in_prj,
                "zUnits": self.z_units,
                "geometry": self.loadedGeoJSON['features'][i]['geometry']
            })
            response = requests.request("POST", url, headers=headers, data=payload)
            self.carsGeoJSON = json.loads(response.text)
            # append GeoJSON with CARS attributes to geoJSONarr 
            self.geoJSONarr.append(self.carsGeoJSON)
        print(f'CARS Data Request Success Flight AGL: {self.agl} {self.z_units}')

    # Process CARS GeoJSON into final GeoJSON with altitude, agl, height_above_takeoff
    def addAglGeoJSON(self):
        '''With geoJSONarr from CARS function, calculate flight altitude
        :return: Prints success message e.g.
        Elevation Calculations Attributes Added
        :rtype: none
        '''       
        self.finalGeoJSON = []
        if type(self.geoJSONarr) is list:
            for i, geoJSON in enumerate(self.geoJSONarr):
                #    
                launchHeight = geoJSON['data']['features'][0]['properties']['terrainWGS84'] 
                for z, feature in enumerate(geoJSON['data']['features']):
                    altitude = feature['properties']['terrainWGS84'] + self.agl
                    geoJSON['data']['features'][z]['properties']['altitude'] = altitude
                    geoJSON['data']['features'][z]['properties']['AGL'] = self.agl
                    geoJSON['data']['features'][z]['properties']['height_above_takeoff'] = round((self.agl + (feature['properties']['terrainWGS84'] - launchHeight)), 2)
                self.finalGeoJSON.append(geoJSON['data'])
        print(f'Elevation Calculations Attributes Added')
    
    # save array of GeoJSON feature collections as KML with formatted attribute table
    def exportToWaypoints(self):
            '''Convert GeoJSON Feature Collection(s) into .waypoints QGroundControl format - version: QGC WPL 110
            The tab separated attributes are in the following order:
            'INDEX', 'CURRENT', 'COORD_FRAME', 'COMMAND', 'PARAM1', 'PARAM2', 'PARAM3', 'PARAM4', 'PARAM5/X/LATITUDE',
            'PARAM6/Y/LONGITUDE', 'PARAM7/Z/ALTITUDE', 'AUTOCONTINUE'
            PARAM1 - PARAM4 can be used to set aircraft flight attributes e.g. pitch, yaw, roll etc...
            :return: Prints success message and named of exported .waypoints file e.g. file saved as: waypointsTest.csv
            :rtype: Pandas DataFrame of flight waypoints with attributes as columns
            '''         
            # check if multiple GeoJSON Feature Collections
            if type(self.finalGeoJSON) is list:
                # iterate through each GeoJSON
                for z, geoJSON in enumerate(self.finalGeoJSON):
                    # Create header dfHead which contains QGC file version: QGC WPL 110
                    cols =['INDEX', 'CURRENT', 'COORD_FRAME', 'COMMAND', 'PARAM1', 'PARAM2','PARAM3','PARAM4',
                    'PARAM5/X/LATITUDE','PARAM6/Y/LONGITUDE','PARAM7/Z/ALTITUDE','AUTOCONTINUE']
                    dfHead = pd.DataFrame(columns=cols)
                    dfHead.loc[-1] = ['QGC', 'WPL', '110', '', '', '', '', '', '', '', '', '']
                    df = pd.DataFrame(columns=cols)
                    # concat the header dataframe and empty dataframe
                    df = pd.concat([df, dfHead])
                    for i, feat in enumerate(geoJSON['features']):
                        # add waypoints as dataframe records
                        # command=22 for aircraft takeoff - command=16 to proceed to next waypoint
                        command = 22 if i == 0 else 16
                        df.loc[i+1] = [i, (i-1), 0, command, 0, 0, 0, 0, feat['geometry']['coordinates'][1],
                                     feat['geometry']['coordinates'][0], feat["properties"]["altitude"], 1]
                    # command=21 for aircraft landing - insert at last waypoint
                    df.at[df.index[-1], 'COMMAND'] = 21
                    # save .waypoints file as tab separated text file
                    df.to_csv(f'{self.outName}.waypoints', sep ='\t', index=False, header=False)
                    print(f'file saved as: {self.outName}.waypoints')
                    return df


## WAYPOINTS PROCESS

In [26]:
# Input parameters
inputData = 'Waypoints_13042022142411.geojson'
z_units = 'm'
agl = 30.48	
outputName = 'waypointsTest'
# Instantiate RouteToKML class
routes = RouteToWaypoints(inputData, z_units, agl, outputName)
# Run Process
routes.runGeoTool()

Obtained token for Client ID: 2D2rcAgjy9cUBXMu226XJ1xzlFuueCAH
Loaded GeoJSON: Waypoints_13042022142411.geojson
CARS Data Request Success Flight AGL: 30.48 m
Elevation Calculations Attributes Added
file saved as: waypointsTest.waypoints


In [10]:
# Input parameters
inputData = 'pendleOR_GeoJSON_13042022132656.geojson'
z_units = 'm'
agl = 30.48	
outputName = 'waypointsTest'
# Instantiate RouteToWaypoints class
routes = RouteToWaypoints(inputData, z_units, agl, outputName)
# Run Process
df = routes.runGeoTool()

Obtained token for Client ID: 2D2rcAgjy9cUBXMu226XJ1xzlFuueCAH
Loaded GeoJSON: pendleOR_GeoJSON_13042022132656.geojson
CARS Data Request Success Flight AGL: 30.48 m
Elevation Calculations Attributes Added
file saved as: waypointsTest.waypoints
